In [ ]:
# to disable GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [1]:
import os
os.chdir('T:/Projects/LSTM-forecast/src')
import featureEng as fe
import processStocks as ps
import lstm
import getStocks
import visualize
import predicts

Using TensorFlow backend.


In [2]:
# view tenslow enabled devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18177862813982931878
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6772842168
locality {
  bus_id: 1
}
incarnation: 16753324186105240408
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:65:00.0, compute capability: 6.1"
]


# User parameters

In [68]:
####TODO: reshaping so we can plot various n_in, n_out
# model seems to work, but cant redim for plot
ticker = 'unh'

n_in = 10
n_out = 5
target = 'Close'

In [69]:
import imp
imp.reload(getStocks)

<module 'getStocks' from 'T:\\Projects\\LSTM-forecast\\src\\getStocks.py'>

In [70]:
# collect a bunch of data from the market, save it to directory
# getStocks.get_mkt_data(source='yahoo')

# Load Data
### Several methods available here:
* Load a single stock
* get a refresh of the stock
* load a single stock as the target while using many other stocks as additional features

In [71]:
# load dataset
#dataset = getStocks.get_single(ticker=ticker, save=True)
# dataset = getStocks.load_single(ticker)
dataset = getStocks.join_tgt_spt(target_ticker='UNH', number_spt=25)
dataset.reset_index(level=0, inplace=True)

#dataset.rename(columns={'Adj Close':'AdjCls'}, inplace=True)

In [72]:
dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume,LEGOpen,LEGHigh,LEGLow,...,FISLow,FISClose,FISAdj Close,FISVolume,GPNOpen,GPNHigh,GPNLow,GPNClose,GPNAdj Close,GPNVolume
0,1984-10-17,0.144531,0.148438,0.144531,0.144531,0.126899,9868800,1.520833,1.520833,1.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1984-10-18,0.148438,0.156250,0.148438,0.148438,0.130329,5324800,1.520833,1.562500,1.520833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1984-10-19,0.148438,0.148438,0.144531,0.144531,0.126899,3043200,1.552083,1.562500,1.552083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1984-10-22,0.144531,0.148438,0.144531,0.144531,0.126899,2326400,1.562500,1.583333,1.562500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1984-10-23,0.144531,0.144531,0.140625,0.140625,0.123469,787200,1.572917,1.572917,1.541667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Feature Engineering
* Add day of week feature
* Move target feature to end of dataframe

In [73]:
## Generate new features
dataset = fe.derivative(dataset, drop_na = True)
dataset = fe.weekDay(dataset)
features = list(dataset.columns)
# move target to position [-1]
features.remove(target)
features.append(target)
features=list(features)
dataset=dataset[features]

In [74]:
# Plot the features
# visualize.plot_features(dataset)

# Format as Supervised Learning Problem

In [75]:
# frame as supervised learning
# this will be for scaling the data to the window scaled_in to scaled_out
reframed=ps.series_to_supervised(dataset, n_in=n_in, n_out=n_out, 
                                   features=features)

starting Series to supverised conversion...


In [76]:
# drop all but the 'target' from the predictor set
# this might be able to take an array for multi-output
reframed=ps.frame_targets(reframed, features, n_out,target=target)

#scaler = MinMaxScaler(feature_range=(-1, 1))
#scaled = scaler.fit_transform(reframed)
# pass the feature in the sequence that we dont want scaled

# Scaling - several methods
* yet to be determined if scaling improves performance

In [77]:
imp.reload(ps)

<module 'processStocks' from 'T:\\Projects\\LSTM-forecast\\src\\processStocks.py'>

In [ ]:
#reframed=reframed.iloc[:10,:] 
#test/debug
#dataset=reframed
scaled=ps.scale_sequence(reframed, 
                         features, 
                         scaleTarget=True, 
                         target=target)

In [79]:
# or skip the scaling
scaled=reframed

In [80]:
#getStocks.saveScaled(scaled, n_in, n_out, ticker)
# load scaled 

#scaled=pd.read_csv('../data/astc/astc_29092017_scaled_30_30.csv')

# Cross Validation Setup

In [81]:
# put in order of time (t-n_in should be on left of df, with t+ on the right)

# split into train, validation, test
train, validation, test = lstm.tscv(scaled, train=0.7, validation=0.25)

In [82]:
# split into input and outputs
# the last n columns are the output variable

train_X, train_y = train[:, :-n_out], train[:, -n_out:]
X_validation, Y_validation = validation[:, :-n_out], validation[:, -n_out:]
test_X, test_y = test[:, :-n_out], test[:, -n_out:]

# *** Test Reshaping Method

In [59]:
# test dimension modification
import pandas as pd
df=pd.read_csv('testDim.csv')
features_test=list(df.columns)
df.head()

,Var0,Var1,Var2,Var3,Var4
0,100,101,102,103,104
1,200,201,202,203,204
2,300,301,302,303,304
3,400,401,402,403,404
4,500,501,502,503,504


In [60]:
n_in=2
n_out=1
tgt_tst='Var4'
df=ps.series_to_supervised(df, 
                           n_in=n_in, 
                           n_out=n_out, 
                           features=features_test)
df.head()

starting Series to supverised conversion...


,Var0(t-02),Var1(t-02),Var2(t-02),Var3(t-02),Var4(t-02),Var0(t-01),Var1(t-01),Var2(t-01),Var3(t-01),Var4(t-01),Var0(t),Var1(t),Var2(t),Var3(t),Var4(t)
2,100.0,101.0,102.0,103.0,104.0,200.0,201.0,202.0,203.0,204.0,300,301,302,303,304
3,200.0,201.0,202.0,203.0,204.0,300.0,301.0,302.0,303.0,304.0,400,401,402,403,404
4,300.0,301.0,302.0,303.0,304.0,400.0,401.0,402.0,403.0,404.0,500,501,502,503,504
5,400.0,401.0,402.0,403.0,404.0,500.0,501.0,502.0,503.0,504.0,600,601,602,603,604
6,500.0,501.0,502.0,503.0,504.0,600.0,601.0,602.0,603.0,604.0,700,701,702,703,704


In [61]:
test_frame=ps.frame_targets(df, 
                          features=features_test, 
                          n_out=n_out,
                          target=tgt_tst)
test_frame.head()

,Var0(t-02),Var1(t-02),Var2(t-02),Var3(t-02),Var4(t-02),Var0(t-01),Var1(t-01),Var2(t-01),Var3(t-01),Var4(t-01),Var4(t)
2,100.0,101.0,102.0,103.0,104.0,200.0,201.0,202.0,203.0,204.0,304
3,200.0,201.0,202.0,203.0,204.0,300.0,301.0,302.0,303.0,304.0,404
4,300.0,301.0,302.0,303.0,304.0,400.0,401.0,402.0,403.0,404.0,504
5,400.0,401.0,402.0,403.0,404.0,500.0,501.0,502.0,503.0,504.0,604
6,500.0,501.0,502.0,503.0,504.0,600.0,601.0,602.0,603.0,604.0,704


In [62]:
X = test_frame.values[:, :-n_out]

In [63]:
[print(x) for x in X]

[ 100.  101.  102.  103.  104.  200.  201.  202.  203.  204.]
[ 200.  201.  202.  203.  204.  300.  301.  302.  303.  304.]
[ 300.  301.  302.  303.  304.  400.  401.  402.  403.  404.]
[ 400.  401.  402.  403.  404.  500.  501.  502.  503.  504.]
[ 500.  501.  502.  503.  504.  600.  601.  602.  603.  604.]
[ 600.  601.  602.  603.  604.  700.  701.  702.  703.  704.]
[ 700.  701.  702.  703.  704.  800.  801.  802.  803.  804.]
[ 800.  801.  802.  803.  804.  900.  901.  902.  903.  904.]
[  900.   901.   902.   903.   904.  1000.  1001.  1002.  1003.  1004.]
[ 1000.  1001.  1002.  1003.  1004.  1100.  1101.  1102.  1103.  1104.]
[ 1100.  1101.  1102.  1103.  1104.  1200.  1201.  1202.  1203.  1204.]
[ 1200.  1201.  1202.  1203.  1204.  1300.  1301.  1302.  1303.  1304.]
[ 1300.  1301.  1302.  1303.  1304.  1400.  1401.  1402.  1403.  1404.]
[ 1400.  1401.  1402.  1403.  1404.  1500.  1501.  1502.  1503.  1504.]
[ 1500.  1501.  1502.  1503.  1504.  1600.  1601.  1602.  1603.  1604.]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [64]:
d=ps.shape(X, n_in, features_test)

In [65]:
d[1,-1,:]

array([ 300.,  301.,  302.,  303.,  304.])

* Seems like dim is (observations, n_in, features), which is correct

* tensor should be (t-2)a, (t-2)b, (t-1)a, (t-1)b, etc. where a and b are features to properly reshape

# Reshape to 3d tenser

In [83]:
scaled.head()

,Open(t-10),High(t-10),Low(t-10),Adj Close(t-10),Volume(t-10),LEGOpen(t-10),LEGHigh(t-10),LEGLow(t-10),LEGClose(t-10),LEGAdj Close(t-10),...,T(t-01),W(t-01),Th(t-01),F(t-01),Close(t-01),Close(t),Close(t+01),Close(t+02),Close(t+03),Close(t+04)
10,122.949997,123.650002,122.220001,118.439194,3269300.0,49.270000,49.490002,49.060001,49.439999,46.025360,...,0.0,1.0,0.0,0.0,125.860001,124.930000,123.849998,122.650002,120.860001,120.610001
11,123.820000,123.879997,121.019997,116.932053,3682300.0,49.560001,49.709999,48.910000,49.200001,45.801933,...,0.0,0.0,1.0,0.0,124.930000,123.849998,122.650002,120.860001,120.610001,119.330002
12,119.599998,123.099998,119.260002,117.268051,6522800.0,48.939999,49.490002,48.730000,49.180000,45.783318,...,0.0,0.0,0.0,1.0,123.849998,122.650002,120.860001,120.610001,119.330002,117.940002
13,122.360001,122.800003,117.809998,115.866516,6476200.0,49.119999,49.500000,48.680000,49.459999,46.043983,...,0.0,0.0,0.0,0.0,122.650002,120.860001,120.610001,119.330002,117.940002,117.879997
14,119.050003,119.989998,118.400002,114.023392,4009800.0,49.060001,49.330002,48.529999,48.639999,45.280609,...,1.0,0.0,0.0,0.0,120.860001,120.610001,119.330002,117.940002,117.879997,121.519997


In [84]:
# ensure the shaping makes sense the way all the variables are arranged
# reshape input to be 3D [samples(observations), timesteps (n_in), features]
train_X = ps.shape(train_X, n_in=n_in, features=features)
X_validation = ps.shape(X_validation, n_in=n_in, features=features)
test_X = ps.shape(test_X, n_in=n_in, features=features)

In [85]:
print(train_X.shape, X_validation.shape, test_X.shape)

(425, 10, 165) (152, 10, 165) (31, 10, 165)


# Build LSTM Architecture

In [86]:
# for debug on lstm package
import imp
imp.reload(lstm)

<module 'lstm' from 'T:\\Projects\\LSTM-forecast\\src\\lstm.py'>

In [87]:
model = lstm.build_model(train_X, 
                         timesteps=n_in, 
                         inlayer=int(train_X.shape[-1]*20),
                         hiddenlayers=[60], 
                         outlayer=n_out)

Compilation Time :  0.031253814697265625
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 3300)          45751200  
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 3300)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60)                806640    
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 305       
Total params: 46,558,145
Trainable params: 46,558,145
Non-trainable params: 0
_________________________________________________________________


# Fit model

In [ ]:
# fit network and save to history
history = model.fit(train_X, train_y, 
                    epochs=150, 
                    batch_size=256, 
                    validation_data=(X_validation, Y_validation), 
                    verbose=2, shuffle=False)

# plot history
visualize.plot_loss(history)

Train on 425 samples, validate on 152 samples
Epoch 1/150


In [ ]:
### KEEP THIS WORKING ####
yhat = predicts.predict_sequences_multiple(model, test_X, n_in, n_out)
visualize.plot_results_multiple(yhat, test_y[:,0], n_out, legend=False)
# visualize.plot_single(yhat, test_y, ticker, data_set_category='test')

In [ ]:
###### TODO METHOD####
#import predicts
## copy weights
#old_weights=model.get_weights()
#
#new_model = Sequential()
#new_model.add(LSTM(30, input_shape=(train_X.shape[1], train_X.shape[2]),
#          return_sequences=True, activation='tanh'))
#new_model.add(Dropout(0.5))
#new_model.add(LSTM(10, activation='tanh'))
#new_model.add(Dropout(0.5))
#new_model.add(Dense(1))
#new_model.compile(loss='mae', optimizer='adam')
## fit network
#new_model.set_weights(old_weights)
#predicts.onlineForecast(new_model, test_X, test_y)
## # # ____ # # #